<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

### Null Hypothesis: Democrats and Republicans vote in the same way

### Alt Hypothesis: Democrats and Republicans **DO NOT** vote in the same way

In [94]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.names

1. Title: 1984 United States Congressional Voting Records Database

2. Source Information:
    (a) Source:  Congressional Quarterly Almanac, 98th Congress, 
                 2nd session 1984, Volume XL: Congressional Quarterly Inc. 
                 Washington, D.C., 1985.
    (b) Donor: Jeff Schlimmer (Jeffrey.Schlimmer@a.gp.cs.cmu.edu)
    (c) Date: 27 April 1987 

3. Past Usage
   - Publications
     1. Schlimmer, J. C. (1987).  Concept acquisition through 
        representational adjustment.  Doctoral dissertation, Department of 
        Information and Computer Science, University of California, Irvine, CA.
        -- Results: about 90%-95% accuracy appears to be STAGGER's asymptote
     - Predicted attribute: party affiliation (2 classes)

4. Relevant Information:
      This data set includes votes for each of the U.S. House of
      Representatives Congressmen on the 16 key votes identified by the
      CQA.  The CQA lists nine different types of votes: voted for, paired
      

In [0]:
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind, ttest_rel

In [0]:



url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
cols = ['party', 'handicapped_infants', 'water_cost_sharing', 'budget', 'doc_fees', 'el_salvador_aid', 'religion_in_school',
       'anti_satellite_test_ban', 'aid_nicaraguan_contras', 'mx_missile', 'immigration', 'synfuels_corp_cuts', 'edu_spending', 'right_to_sue', 'crime', 'duty_free_export', 'admin_act_s_africa']
df = pd.read_csv(url, header=None, names=cols, na_values='?')

In [97]:
print(df.shape)
df.head()

(435, 17)


,party,handicapped_infants,water_cost_sharing,budget,doc_fees,el_salvador_aid,religion_in_school,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cuts,edu_spending,right_to_sue,crime,duty_free_export,admin_act_s_africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y


In [98]:
df.isna().sum()

party                        0
handicapped_infants         12
water_cost_sharing          48
budget                      11
doc_fees                    11
el_salvador_aid             15
religion_in_school          11
anti_satellite_test_ban     14
aid_nicaraguan_contras      15
mx_missile                  22
immigration                  7
synfuels_corp_cuts          21
edu_spending                31
right_to_sue                25
crime                       17
duty_free_export            28
admin_act_s_africa         104
dtype: int64

In [99]:
df.describe()

,party,handicapped_infants,water_cost_sharing,budget,doc_fees,el_salvador_aid,religion_in_school,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cuts,edu_spending,right_to_sue,crime,duty_free_export,admin_act_s_africa
count,435,423,387,424,424,420,424,421,420,413,428,414,404,410,418,407,331
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,democrat,n,y,y,n,y,y,y,y,y,y,n,n,y,y,n,y
freq,267,236,195,253,247,212,272,239,242,207,216,264,233,209,248,233,269


In [0]:
democrat = df[df['party'] == 'democrat']
republican = df[df['party'] == 'republican']

In [101]:
print(democrat.shape)
print(democrat.describe())
democrat.head()

(267, 17)
           party handicapped_infants  ... duty_free_export admin_act_s_africa
count        267                 258  ...              251                185
unique         1                   2  ...                2                  2
top     democrat                   y  ...                y                  y
freq         267                 156  ...              160                173

[4 rows x 17 columns]


,party,handicapped_infants,water_cost_sharing,budget,doc_fees,el_salvador_aid,religion_in_school,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cuts,edu_spending,right_to_sue,crime,duty_free_export,admin_act_s_africa
2,democrat,NaN,y,y,NaN,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,NaN,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,NaN,y,y,y,y
5,democrat,n,y,y,n,y,y,n,n,n,n,n,n,y,y,y,y
6,democrat,n,y,n,y,y,y,n,n,n,n,n,n,NaN,y,y,y


In [102]:
print(republican.shape)
print(republican.describe())
republican.head()

(168, 17)
             party handicapped_infants  ... duty_free_export admin_act_s_africa
count          168                 165  ...              156                146
unique           1                   2  ...                2                  2
top     republican                   n  ...                n                  y
freq           168                 134  ...              142                 96

[4 rows x 17 columns]


,party,handicapped_infants,water_cost_sharing,budget,doc_fees,el_salvador_aid,religion_in_school,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cuts,edu_spending,right_to_sue,crime,duty_free_export,admin_act_s_africa
0,republican,n,y,n,y,y,y,n,n,n,y,NaN,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,NaN
7,republican,n,y,n,y,y,y,n,n,n,n,n,n,y,y,NaN,y
8,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,y
10,republican,n,y,n,y,y,n,n,n,n,n,NaN,NaN,y,y,n,n


In [103]:
mapping = {'n': 0, 'y': 1}

for col in democrat.columns:
  if col != 'party':
    democrat.replace({col: mapping}, inplace=True)
  

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [104]:
for col in republican.columns:
  if col != 'party':
    republican.replace({col: mapping}, inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  regex=regex)


In [105]:
democrat.head()

,party,handicapped_infants,water_cost_sharing,budget,doc_fees,el_salvador_aid,religion_in_school,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cuts,edu_spending,right_to_sue,crime,duty_free_export,admin_act_s_africa
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,1.0,1.0,1.0


In [106]:
republican.head()

,party,handicapped_infants,water_cost_sharing,budget,doc_fees,el_salvador_aid,religion_in_school,anti_satellite_test_ban,aid_nicaraguan_contras,mx_missile,immigration,synfuels_corp_cuts,edu_spending,right_to_sue,crime,duty_free_export,admin_act_s_africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
7,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,NaN,1.0
8,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0
10,republican,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,1.0,1.0,0.0,0.0


In [0]:
import random
def fillnan(df):
  for col in df.columns:
    df[col].fillna(random.randint(0,1), inplace=True)
  return df    

In [108]:
democrat = fillnan(democrat)
republican = fillnan(republican)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6130: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [109]:
print(democrat.isna().sum())
print(republican.isna().sum())

party                      0
handicapped_infants        0
water_cost_sharing         0
budget                     0
doc_fees                   0
el_salvador_aid            0
religion_in_school         0
anti_satellite_test_ban    0
aid_nicaraguan_contras     0
mx_missile                 0
immigration                0
synfuels_corp_cuts         0
edu_spending               0
right_to_sue               0
crime                      0
duty_free_export           0
admin_act_s_africa         0
dtype: int64
party                      0
handicapped_infants        0
water_cost_sharing         0
budget                     0
doc_fees                   0
el_salvador_aid            0
religion_in_school         0
anti_satellite_test_ban    0
aid_nicaraguan_contras     0
mx_missile                 0
immigration                0
synfuels_corp_cuts         0
edu_spending               0
right_to_sue               0
crime                      0
duty_free_export           0
admin_act_s_africa         0
d

In [110]:
print('Democrats: {}'.format(democrat['handicapped_infants'].value_counts()))

print('Republicans: {}'.format(republican['handicapped_infants'].value_counts()))

Democrats: 1.0    165
0.0    102
Name: handicapped_infants, dtype: int64
Republicans: 0.0    134
1.0     34
Name: handicapped_infants, dtype: int64


In [111]:
print(democrat['handicapped_infants'].mean())

print(republican['handicapped_infants'].mean())

0.6179775280898876
0.20238095238095238


In [112]:
ttest_ind(democrat['handicapped_infants'], republican['handicapped_infants'])

Ttest_indResult(statistic=9.24894185392722, pvalue=1.0436856317093103e-18)

In [113]:
tsts = []
pvals = []
for col in democrat.columns:
  if col != 'party':
    tstat, pval = ttest_ind(democrat[col], republican[col])
    tsts.append(tstat)
    pvals.append(pval)
    print(col + ' T-Statistic: ' + str(tstat))
    print(col + ' P-Value: ' + str(pval))
    print('----------------------------')

handicapped_infants T-Statistic: 9.24894185392722
handicapped_infants P-Value: 1.0436856317093103e-18
----------------------------
water_cost_sharing T-Statistic: -2.366543363839921
water_cost_sharing P-Value: 0.018394291752627714
----------------------------
budget T-Statistic: 22.5072082955668
budget P-Value: 7.529526421191217e-75
----------------------------
doc_fees T-Statistic: -50.641376160580805
doc_fees P-Value: 4.976027685195905e-184
----------------------------
el_salvador_aid T-Statistic: -22.022186479520858
el_salvador_aid P-Value: 1.168664184293552e-72
----------------------------
religion_in_school T-Statistic: -9.435976590742047
religion_in_school P-Value: 2.39864396592911e-19
----------------------------
anti_satellite_test_ban T-Statistic: 12.201859703940068
anti_satellite_test_ban P-Value: 1.2271674100823548e-29
----------------------------
aid_nicaraguan_contras T-Statistic: 18.28795984953887
aid_nicaraguan_contras P-Value: 8.889372685886026e-56
---------------------

In [114]:
tests = pd.DataFrame({'t_stats': tsts, 'p_value': pvals}, index=democrat.columns[1:])
tests

,t_stats,p_value
handicapped_infants,9.248942,1.043686e-18
water_cost_sharing,-2.366543,1.839429e-02
budget,22.507208,7.529526e-75
doc_fees,-50.641376,4.976028e-184
el_salvador_aid,-22.022186,1.168664e-72
religion_in_school,-9.435977,2.398644e-19
anti_satellite_test_ban,12.201860,1.227167e-29
aid_nicaraguan_contras,18.287960,8.889373e-56
mx_missile,16.805298,3.725423e-49
immigration,-1.691439,9.147245e-02


In [115]:
voting_means = pd.DataFrame([democrat.mean(), republican.mean()], index=['democrat', 'republican'])
voting_means = voting_means.T
voting_means

,democrat,republican
handicapped_infants,0.617978,0.202381
water_cost_sharing,0.449438,0.565476
budget,0.891386,0.154762
doc_fees,0.052434,0.988095
el_salvador_aid,0.205993,0.952381
religion_in_school,0.494382,0.898810
anti_satellite_test_ban,0.749064,0.232143
aid_nicaraguan_contras,0.816479,0.142857
mx_missile,0.775281,0.130952
immigration,0.464419,0.547619


In [116]:
mean_diff = voting_means['democrat'] - voting_means['republican']
mean_diff

handicapped_infants        0.415597
water_cost_sharing        -0.116038
budget                     0.736624
doc_fees                  -0.935661
el_salvador_aid           -0.746388
religion_in_school        -0.404428
anti_satellite_test_ban    0.516921
aid_nicaraguan_contras     0.673622
mx_missile                 0.644329
immigration               -0.083200
synfuels_corp_cuts         0.304575
edu_spending              -0.601324
right_to_sue              -0.536116
crime                     -0.607611
duty_free_export           0.515918
admin_act_s_africa         0.076512
dtype: float64

In [0]:
frame = voting_means.join(tests, how='outer')

In [118]:
frame

,democrat,republican,t_stats,p_value
handicapped_infants,0.617978,0.202381,9.248942,1.043686e-18
water_cost_sharing,0.449438,0.565476,-2.366543,1.839429e-02
budget,0.891386,0.154762,22.507208,7.529526e-75
doc_fees,0.052434,0.988095,-50.641376,4.976028e-184
el_salvador_aid,0.205993,0.952381,-22.022186,1.168664e-72
religion_in_school,0.494382,0.898810,-9.435977,2.398644e-19
anti_satellite_test_ban,0.749064,0.232143,12.201860,1.227167e-29
aid_nicaraguan_contras,0.816479,0.142857,18.287960,8.889373e-56
mx_missile,0.775281,0.130952,16.805298,3.725423e-49
immigration,0.464419,0.547619,-1.691439,9.147245e-02



1) More Democrats supported passing the budget in 1980 than Republicans, with a p-value of 7.529526e-75

2) More Republicans supported passing the crime bill than Democrats with a p-value of 5.171316e-45

3) The only issue with a p-value greater than 0.1 is the Export Act with South Africa with  a p-value of 0.1102368


In [120]:
1.102368e-01 > 0.1

True